# Sprint 機械学習スクラッチ入門

## スクラッチの意義
ここでのスクラッチとは、NumPyなどの基本的なライブラリを組み合わせることで、scikit-learnのような応用的なライブラリと同じ機能のクラス・関数を自作することを指します。

スクラッチをすることでscikit-learnなどのライブラリを動かすだけでは掴みづらい、アルゴリズムの深い理解を目指します。コーディングのスキル向上も兼ねますが、それは主な目的ではありません。

以下のような効果を狙っています。

- 新たな手法に出会った時に理論・数式を理解しやすくする
- ライブラリを使う上での曖昧さを減らす
- 既存の実装を読みやすくする

## 【問題1】train_test_splitのスクラッチ
スクラッチの練習として、scikit-learnのtrain_test_splitを自作してみます。以下の雛形をベースとして関数を完成させてください。

[sklearn.model_selection.train_test_split — scikit-learn 0.21.3 documentation](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

なお、作成した関数がscikit-learnのtrain_test_splitと同じ動作をしているか必ず確認をするようにしましょう。

In [1]:
import numpy as np
from math import ceil, floor
import random
def scratch_train_test_split(X, y, train_size=0.8,):
    """
    検証データを分割する。

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, )
      正解値
    train_size : float (0<train_size<1)
      何割をtrainとするか指定

    Returns
    ----------
    X_train : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    X_test : 次の形のndarray, shape (n_samples, n_features)
      検証データ
    y_train : 次の形のndarray, shape (n_samples, )
      訓練データの正解値
    y_test : 次の形のndarray, shape (n_samples, )
      検証データの正解値
    """
    if type(X) != np.ndarray :
        raise TypeError("Type of X input must be numpy.array")
    elif type(y) != np.ndarray:
        raise TypeError("Type of y input must be numpy.array")
    elif len(X) == 0 or len(y) == 0:
        raise ValueError("At least one array required as input")
    elif X.shape[0] != y.shape[0]:
        raise SyntaxError('The number of rows in the array must be match')
    elif type(train_size) != float:
        raise TypeError("Type of train_size input must be float")
    elif train_size >= 1 or train_size <= 0:
        raise ValueError('Value of train_size must be between 0 and 1')
    n_samples = len(X)
    n_train = int(floor(train_size * n_samples))
    n_test = int(n_samples - n_train)
    if n_train == 0:
        raise ValueError(
            'With n_samples={}, test_size={} and train_size={}, the '
            'resulting train set will be empty. Adjust any of the '
            'aforementioned parameters.'.format(n_samples, test_size,
                                                train_size)
        )
    index_lst = [i for i in range(n_samples)]
    test_index = random.sample(index_lst, n_test)
    train_index = list(set(index_lst)-set(test_index))
    X_train = X[train_index]
    y_train = y[train_index]
    X_test = X[test_index]
    y_test = y[test_index]
    return X_train, X_test, y_train, y_test

In [2]:
X = np.arange(150).reshape(15, -1)
y = np.arange(15)
X_train, X_test, y_train, y_test = scratch_train_test_split(X, y, train_size=0.8)
print(X_test, y_test)
print(len(X_train),len(X_test),len(y_train),len(y_test))

[[140 141 142 143 144 145 146 147 148 149]
 [110 111 112 113 114 115 116 117 118 119]
 [  0   1   2   3   4   5   6   7   8   9]] [14 11  0]
12 3 12 3


In [3]:
# sklearn で確認
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)
print(X_test, y_test)
print(len(X_train),len(X_test),len(y_train),len(y_test))

[[ 20  21  22  23  24  25  26  27  28  29]
 [130 131 132 133 134 135 136 137 138 139]
 [  0   1   2   3   4   5   6   7   8   9]] [ 2 13  0]
12 3 12 3


## 分類問題
分類は3種類の手法をスクラッチします。

- ロジスティック回帰
- SVM
- 決定木

ロジスティック回帰はscikit-learnにおいてLogisticRegressionクラスとSGDClassifierクラスの2種類から使用できます。ここでは勾配降下法を用いて計算するSGDClassifierクラスを利用してください。引数で`loss="log"`とすることでロジスティック回帰の計算になります。

- [sklearn.linear_model.SGDClassifier — scikit-learn 0.21.3 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html#sklearn.linear_model.SGDClassifier)
- [sklearn.svm.SVC — scikit-learn 0.21.3 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC)
- [sklearn.tree.DecisionTreeClassifier — scikit-learn 0.21.3 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier)

データセットは3種類用意します。

1つ目は事前学習期間同様にirisデータセットです。

[sklearn.datasets.load_iris — scikit-learn 0.20.2 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html)

2値分類としたいため、以下の2つの目的変数のみ利用します。特徴量は4種類全て使います。

- virgicolorとvirginica

残り2つは特徴量が2つのデータセットを人工的に用意します。以下のコードで説明変数`X`,目的変数`y`が作成可能です。「シンプルデータセット1」「シンプルデータセット2」とします。特徴量が2つであるため可視化が容易です。

In [4]:
from sklearn.datasets import load_iris
import pandas as pd
data = load_iris()
names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
iris_df = pd.DataFrame(data.data, columns=names)
iris_df['Species'] = data.target
iris_df_choice = iris_df[(iris_df['Species'] == 1) | (iris_df['Species'] == 2)]
X = iris_df_choice.iloc[:, 0:4].values
y = iris_df_choice.iloc[:, -1].values
iris_data = X, y

In [5]:
# シンプルデータセット1作成コード
import numpy as np
np.random.seed(seed=0)
n_samples = 500
f0 = [-1, 2]
f1 = [2, -1]
cov = [[1.0,0.8], [0.8, 1.0]]
f0 = np.random.multivariate_normal(f0, cov, int(n_samples/2))
f1 = np.random.multivariate_normal(f1, cov, int(n_samples/2))
X = np.concatenate((f0, f1))
y = np.concatenate((np.ones((int(n_samples/2))), np.ones((int(n_samples/2))) *(-1))).astype(np.int)
random_index = np.random.permutation(np.arange(n_samples))
X = X[random_index]
y = y[random_index]

In [6]:
simple_data1 = X, y

In [7]:
# シンプルデータセット2作成コード
X = np.array([[-0.44699 , -2.8073  ],[-1.4621  , -2.4586  ],
       [ 0.10645 ,  1.9242  ],[-3.5944  , -4.0112  ],
       [-0.9888  ,  4.5718  ],[-3.1625  , -3.9606  ],
       [ 0.56421 ,  0.72888 ],[-0.60216 ,  8.4636  ],
       [-0.61251 , -0.75345 ],[-0.73535 , -2.2718  ],
       [-0.80647 , -2.2135  ],[ 0.86291 ,  2.3946  ],
       [-3.1108  ,  0.15394 ],[-2.9362  ,  2.5462  ],
       [-0.57242 , -2.9915  ],[ 1.4771  ,  3.4896  ],
       [ 0.58619 ,  0.37158 ],[ 0.6017  ,  4.3439  ],
       [-2.1086  ,  8.3428  ],[-4.1013  , -4.353   ],
       [-1.9948  , -1.3927  ],[ 0.35084 , -0.031994],
       [ 0.96765 ,  7.8929  ],[-1.281   , 15.6824  ],
       [ 0.96765 , 10.083   ],[ 1.3763  ,  1.3347  ],
       [-2.234   , -2.5323  ],[-2.9452  , -1.8219  ],
       [ 0.14654 , -0.28733 ],[ 0.5461  ,  5.8245  ],
       [-0.65259 ,  9.3444  ],[ 0.59912 ,  5.3524  ],
       [ 0.50214 , -0.31818 ],[-3.0603  , -3.6461  ],
       [-6.6797  ,  0.67661 ],[-2.353   , -0.72261 ],
       [ 1.1319  ,  2.4023  ],[-0.12243 ,  9.0162  ],
       [-2.5677  , 13.1779  ],[ 0.057313,  5.4681  ]])
y = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [8]:
simple_data2 = X, y

## 【問題2】 分類問題を解くコードの作成
上記3種類の手法で3種類のデータセットを学習・推定するコードを作成してください。

In [9]:
# ロジスティック回帰（勾配降下法）
def scratch_SGDClassifier(X, y, loss='log'):
    X_train, X_test, y_train, y_test = scratch_train_test_split(X, y)
    from sklearn.linear_model import SGDClassifier 
    clf = SGDClassifier(loss=loss)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return y_pred

In [10]:
# SVM
def scratch_SVC(X, y):
    X_train, X_test, y_train, y_test = scratch_train_test_split(X, y)
    from sklearn.svm import SVC
    clf = SVC()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return y_pred

In [11]:
# 決定木
def scratch_DecisionTreeClassifier(X, y):
    X_train, X_test, y_train, y_test = scratch_train_test_split(X, y)
    from sklearn.tree import DecisionTreeClassifier
    clf = DecisionTreeClassifier()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return y_pred

In [12]:
data_lst = [iris_data, simple_data1, simple_data2]
for data in data_lst:
    print('ロジスティック回帰（勾配降下法）', scratch_SGDClassifier(*data), sep='\n')
    print('SVM', scratch_SVC(*data), sep='\n')
    print('決定木', scratch_DecisionTreeClassifier(*data), sep='\n')

ロジスティック回帰（勾配降下法）
[2 2 2 2 2 2 2 2 1 2 1 1 2 2 2 1 1 1 2 2]
SVM
[2 2 1 2 2 1 1 2 1 1 1 2 1 2 2 1 1 1 1 1]
決定木
[2 2 1 1 2 1 2 2 1 2 1 2 2 1 1 2 1 1 1 2]
ロジスティック回帰（勾配降下法）
[-1  1  1  1  1 -1  1  1 -1  1 -1  1  1 -1  1  1 -1 -1  1  1  1 -1  1 -1
 -1 -1  1  1 -1  1  1  1  1  1 -1 -1 -1  1  1 -1  1 -1 -1  1  1  1  1 -1
 -1  1  1 -1 -1 -1 -1  1 -1 -1  1 -1  1 -1 -1 -1 -1  1  1 -1  1 -1 -1 -1
  1  1 -1 -1  1  1 -1  1 -1 -1  1  1 -1  1  1 -1  1  1 -1  1  1  1  1  1
  1  1  1 -1]
SVM
[-1  1 -1 -1 -1  1 -1 -1 -1  1 -1 -1  1 -1  1 -1  1  1 -1 -1 -1  1 -1 -1
 -1  1 -1  1  1  1 -1  1 -1  1 -1  1 -1 -1  1  1 -1 -1  1 -1  1 -1  1  1
  1 -1  1  1 -1 -1  1  1  1  1  1  1  1 -1 -1  1 -1  1 -1 -1  1 -1 -1 -1
  1 -1  1 -1  1 -1 -1  1  1  1 -1 -1  1  1  1 -1  1 -1  1  1  1 -1  1 -1
  1 -1 -1 -1]
決定木
[-1 -1 -1  1 -1  1 -1 -1 -1  1  1 -1 -1 -1  1 -1  1 -1  1 -1 -1  1 -1 -1
 -1 -1 -1  1 -1  1 -1  1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1  1 -1 -1 -1 -1
  1 -1 -1 -1 -1 -1 -1 -1  1  1  1 -1 -1 -1  1 -1 -1 -1 -1 -1  1 -1

/Users/ikeda/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/ikeda/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/ikeda/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning."

## 回帰問題
回帰は1種類をスクラッチします。

- 線形回帰

線形回帰は勾配降下法を用いて計算するSGDRegressorクラスを利用してください。

[sklearn.linear_model.SGDRegressor — scikit-learn 0.21.3 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html)

データセットは事前学習期間同様にHouse Pricesコンペティションのものを使います。

[House Prices: Advanced Regression Techniques](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)

`train.csv`をダウンロードし、目的変数として`SalePrice`、説明変数として、`GrLivArea`と`YearBuilt`を使います。

In [13]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", 80)
df = pd.read_csv('train.csv', index_col= 'Id')
X = df.loc[:, ['GrLivArea', 'YearBuilt']].values
y = df.loc[:, 'SalePrice'].values
House_Prices_data = X, y

## 【問題3】 回帰問題を解くコードの作成
線形回帰でHouse Pricesデータセットを学習・推定するコードを作成してください。

In [14]:
# 線形回帰（勾配降下法）
def scratch_SGDRegressor(X, y):
    X_train, X_test, y_train, y_test = scratch_train_test_split(X, y)
    from sklearn.linear_model import SGDRegressor
    clf = SGDRegressor()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return y_pred

In [15]:
print('線形回帰（勾配降下法）', scratch_SGDRegressor(*House_Prices_data), sep='\n')

線形回帰（勾配降下法）
[1.87551482e+15 1.29493184e+15 2.44342771e+15 1.64746524e+15
 2.13412575e+15 2.12189054e+15 2.75107286e+15 1.88992708e+15
 1.79469332e+15 2.25207576e+15 2.02358864e+15 2.30191037e+15
 1.83444175e+15 1.83049719e+15 1.95936845e+15 2.01094656e+15
 2.47403708e+15 1.94584728e+15 1.75931631e+15 2.12190521e+15
 2.41241547e+15 2.23024927e+15 1.68678280e+15 1.60510221e+15
 1.72260401e+15 2.40239199e+15 1.60902543e+15 1.62518920e+15
 2.10135534e+15 1.60946164e+15 2.18089889e+15 1.99783626e+15
 2.17825762e+15 2.19227769e+15 2.40588833e+15 2.37179996e+15
 2.59242997e+15 2.03060534e+15 2.41593716e+15 1.52733284e+15
 2.87375582e+15 2.38794638e+15 1.59198791e+15 1.91178825e+15
 1.95590545e+15 2.53607089e+15 1.81126795e+15 3.11059495e+15
 1.91525658e+15 2.44823269e+15 2.24421065e+15 1.94455199e+15
 2.37884066e+15 2.50852166e+15 2.33027601e+15 1.77938196e+15
 2.32066739e+15 2.95064210e+15 2.47007785e+15 2.03018647e+15
 1.98737256e+15 2.14770294e+15 1.99696384e+15 1.53128407e+15
 2.20008945e